## Robot navigation 

**This file gives a simple robot navigation case study**

### Initialize the Robot Dynamics

In [7]:
# load dynamics
from Demos.demo_Models import simple_robot
import polytope as pc
import numpy as np
from best.logic.translate import formula_to_pomdp

Robot = simple_robot()  # type: 




Robot.input_space = pc.box2poly(np.kron(np.ones((Robot.m, 1)), np.array([[-1, 1]])))  # continuous set of inputs
Robot.state_space = pc.box2poly(np.kron(np.ones((Robot.dim, 1)), np.array([[-10, 10]])))  # X space



Load robot model: Simple integrator model
[[0.31623 0.     ]
 [0.      0.31623]]


### Grid Robot

In [8]:



print("-------Grid the robots state space------")
#  set the state spaces:
from Reduce.Gridding import grid
from Controller.Approx_sim import rel_eps_del
from best.models.pomdp import POMDP, POMDPNetwork
from best.models.pomdp_sparse_utils import get_T_uxXz, diagonal
from best.solvers.occupation_lp_new import *

d_opt = np.array([[0.69294], [0.721]]) # tuned gridding ratio from a previous paper
d = 2* d_opt  # with distance measure 0.6=default
un = 7


## real computations
Ms, srep = grid(Robot, d, un=un)

network = POMDPNetwork()
network.add_pomdp(POMDP( Ms.transition, input_names=['a'], state_name='s'))



print('did not compute eps and del')
rel = rel_eps_del(Ms, Robot)

eps, M, K, status = rel.get_eps()
print("Status of epsilon computation: ",status)
print('set eps to =', eps)



-------Grid the robots state space------
------- start gridding -------
did not compute eps and del
=========warning===========
interface variable K is seen as an optimization variable
LMI_set B [[1. 0.]
 [0. 1.]]
    This function computes an invariant set R  for the system
initiated variables
status: optimal
optimal epsilon [[1.00007]]
optimal M [[ 1.00008 -0.     ]
 [-0.       1.00008]]
Optimal K [[-1. -0.]
 [-0. -1.]]
Status of epsilon computation:  optimal
set eps to = [[1.00007]]


In [13]:
from best.models.lti import LTI, LTIGaussian

from best.abstraction.gridding import LTIGrid

A = np.eye(2) 
B = np.eye(2)
W = np.array([[.4,-0.2],[-0.2,0.4]])   
C = np.array([[1, 0],[0,1]])
sys_lti = LTI(A, B, C, None, W=W)
# Define spaces
sys_lti.setU(pc.box2poly(np.kron(np.ones((sys_lti.m, 1)), np.array([[-1, 1]])))) # continuous set of inputs
sys_lti.setX(pc.box2poly(np.kron(np.ones((sys_lti.dim, 1)), np.array([[-10, 10]])))) # X space

lti_belief_n = sys_lti.normalize()

abstr = LTIGrid(lti_belief_n, d.flatten(), un=3 )
print ('before prune: nnz: {}, sparsity: {}'.format(abstr.mdp.nnz, abstr.mdp.sparsity))
abstr.mdp.prune(1e-6)
print ('after prune: nnz: {}, sparsity: {}'.format(abstr.mdp.nnz, abstr.mdp.sparsity))

status: optimal
optimal epsilon 2.000078223117768
optimal M [[ 1.00005 -0.     ]
 [-0.       1.00005]]
Optimal K [[ 0.70704 -0.     ]
 [-0.      -0.70704]]
before prune: nnz: 645310, sparsity: 0.40454020859231843
after prune: nnz: 63367, sparsity: 0.03972431761148819


### Specify the required behaviour

In [ ]:
from Demos.Robot_navigation import specify_robot
from Controller.Specifications import Fsa

# specify required behaviour
formula, regions = specify_robot()  # type: dict


fsaform = Fsa() # this is a FSA , contains a digraph
fsaform.from_formula(formula) # given a more complex formula

print("The synthesised formula is:",formula)
print("The FSA is :", fsaform)

dfsa, dfsa_init, dfsa_final = formula_to_pomdp(formula)
print(dfsa)
network.add_pomdp(dfsa)



In [ ]:
print(network)


In [ ]:
import copy
props={'col': 1, 'pac': 2, 'obst': 4}

s = 0

 
    
val = sum([vals * int(Ms.output_space[s] in pc.Region(regions[key])) for key,vals in props.items()])
    
print (val)
copy_ouputmap = copy.deepcopy(Ms.output_space)
func =  lambda s: set([int(Ms.output_space[s] in pc.Region(regions[key]))])
for key in props:
    # network.add_connection(['s'], key, func )
    
    network.add_connection(['s'], key, lambda s_xc: Ms.polytopic_predicate(s_xc[1], regions[key]) )
 


### Plot the layout of the specifications on finite MDP
- empty circle = potential regions of interest
	Computed by dilating the original polytope
- full circles = definite regions of interest (e.g., obstacle, col, pac)
	Computed by eroding the original polytopes

In [ ]:
# show specification on gridded robot
%matplotlib inline 
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.gca()
# Make data.
X = srep[0]
Y = srep[1]
Z = []
Obs_ero = pc.erode(pc.Region(regions['obst']),eps)
Obs_dil = pc.dilate(pc.Region(regions['obst']),eps)
Pac_ero = pc.erode(pc.Region(regions['pac']),eps)
Pac_dil = pc.dilate(pc.Region(regions['pac']),eps)
Col_ero = pc.erode(pc.Region(regions['col']),eps)
Col_dil = pc.dilate(pc.Region(regions['col']),eps)

for i in range(len(Ms.output_space)):
    if np.isnan(Ms.output_space[i]).any():
        continue
    if Ms.output_space[i] in Obs_ero:
        plt.plot(Ms.state_space[i][0], Ms.state_space[i][1],'bo')
        
    elif Ms.output_space[i] in Obs_dil:
        plt.plot(Ms.state_space[i][0], Ms.state_space[i][1],'bo', mfc='none')

    if Ms.output_space[i] in Pac_ero:
        plt.plot(Ms.state_space[i][0], Ms.state_space[i][1], 'go')
        
    elif Ms.output_space[i] in Pac_dil:
        plt.plot(Ms.state_space[i][0], Ms.state_space[i][1], 'go', mfc='none')
        
    if Ms.output_space[i] in Col_ero:
        plt.plot(Ms.state_space[i][0], Ms.state_space[i][1], 'ro')
        
    elif Ms.output_space[i] in Col_dil:
        plt.plot(Ms.state_space[i][0], Ms.state_space[i][1], 'ro', mfc='none')

plt.show()





In [ ]:
import copy


network.add_connection(['s'], 'l', 
                       lambda s: set([1]) 
                       if (Ms._state_space[s][0] > 4) 
                       else set([0]))
accept = np.zeros((Ms.transition.shape[1], 2))
accept[:, 1] = 1


In [ ]:
Robotmdp = diagonal(get_T_uxXz(network.pomdps['s']), 2, 3)

Robotdfa = diagonal(get_T_uxXz(network.pomdps['q']), 2, 3)
print(Robotdfa.todense())


conn = network.connections[0][2]

delta = 0.01


In [ ]:
# reach_prob, val2 = solve_delta(Robotmdp, Robotdfa, conn, delta, s0=0, q0=0, q_target=1)

strat = np.array([[0] * (Robotmdp.shape[2]), [1] * (Robotmdp.shape[2])]).transpose()
strat = strat[conn.transpose()]

model,reach_prob = solve_ltl(Robotmdp, Robotdfa, strat,0, s0=0, q0=0, q_target=1)


 

In [ ]:
# synthesize control
from Controller.Robust import Robust_control

ro_pol = Robust_control(Ms, fsaform, regions, eps.item(), 0, M=None)
ro_pol.init_V()

number_of_backups = 40
for i in range(number_of_backups):
    ro_pol.back_up() # do 40
    





In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
%matplotlib inline  

ro_pol._q_back_up()   
# ax = fig.gca(projection='3d')
# Make data.
X = srep[0]
Y = srep[1]
q_it= ro_pol.q_it.copy()+ list(ro_pol.q_final)

q = q_it.pop()
print("inital", ro_pol.q_init)
print("goal", ro_pol.q_final)
print("trap", ro_pol.q_end)




# Plot the surface.
ax1 = plt.subplot(221)
ro_pol.plot(q)

#plt.pcolor(X, Y, Z,vmin=0, vmax=1)
# plt.colorbar(surf)


# Customize the z axis.
# ax.set_xlim(srep[0][0], srep[0][-1])
# ax.set_ylim(srep[1][0], srep[1][-1])

# plt.setp(ax1.get_xticklabels(), fontsize=6)

# share x only
ax2 = plt.subplot(222, sharex=ax1,sharey=ax1)
q = q_it.pop()
ro_pol.plot(q)
# Z = ro_pol.V[:-1,q]
# Z = Z.reshape(X.shape, order='F')
# plt.pcolor(X, Y, Z,vmin=0, vmax=1)
# # plt.colorbar()

# make these tick labels invisible
# plt.setp(ax2.get_xticklabels(), visible=False)

# share x and y
ax3 = plt.subplot(223, sharex=ax1, sharey=ax1)
q = q_it.pop()
ro_pol.plot(q)

# print(q)
# 
# Z = ro_pol.V[:-1,q]
# Z = Z.reshape(X.shape, order='F')
# plt.pcolor(X, Y, Z,vmin=0, vmax=1)
#plt.colorbar()

# share x and y
ax4 = plt.subplot(224, sharex=ax1, sharey=ax1)
q = q_it.pop()
ro_pol.plot(q)

# print(q)
# 
# Z = ro_pol.V[:-1,q]
# Z = Z.reshape(X.shape, order='F')
# plt.pcolor(X, Y, Z,vmin=0, vmax=1)
#plt.colorbar()

plt.subplots_adjust(top=0.92, bottom=0.08, left=0.10, right=0.95, hspace=0.25,
                    wspace=0.35)
cax = plt.axes([.99, 0.05, 0.02, .9])
plt.colorbar(cax=cax)

#plt.show()


# ax.zaxis.set_major_locator(LinearLocator(10))
# ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

# Add a color bar which maps values to colors.

print("Difference with paper is caused by a factor 2 differences in epsilon. "
      "This is  related to the way we grid there. ")

### Get the robust policy 

In [ ]:
from Controller.Robust import interface
ro_pol.get_pol()
int_f = interface(ro_pol,rel)


### Run simulation of robust policy on original model

In [ ]:

print("------------- Start simulation -------------")
print("--> 1. Reset current states ")
x = np.array([[-8],[-8]])
int_f.reset(x)

print("--> 2. Start simulation ")
time = 90
x_seq = []
q_seq = []

u_seq = []
y_seq = []
print("dead states, ", ro_pol.q_end)
print("final states, ", ro_pol.q_final)
print("initial states, ", ro_pol.q_init)

for t in range(time):
    q_seq += [int_f.q]
    x_seq += [x]
    y_seq += [Robot.h(x)]
    u = int_f.u(x)
    u_seq += [u]
    x = Robot.next_x( x,u)
    if int_f.q in ro_pol.q_final:
        break
        
    
 
X = srep[0]
Y = srep[1]
q_it= ro_pol.q_it.copy()+ list(ro_pol.q_final)

 
X, Y = np.meshgrid(X, Y)


x = np.concatenate(x_seq,axis=1)
# fig = plt.figure()
# ax = fig.add_subplot(111)
Z = ro_pol.V[:-1,q_seq[0]]
Z = Z.reshape(X.shape, order='F')

plt.pcolor(X, Y, Z,vmin=0, vmax=1)

plt.plot(x[0],x[1])
plt.scatter(x[0],x[1], label='Finite states', color='k', s=10, marker="o")
plt.xlabel('x_1') 
plt.ylabel('x_2') 
plt.xlim(np.array([-15,15]))
plt.ylim(np.array([-15,15]))
    
    


In [ ]:
from Reduce.Gridding import grid

d_opt = np.array([[0.69294], [0.721]]) # tuned gridding ratio from a previous paper
 

d = 1.1 * d_opt  # with distance measure 0.6=default
un = 3

Ms, srep  = grid(Robot, d, un=un)

print('# states')
print(len(Ms._state_space))
network = POMDPNetwork()

T00 = Ms.transition

network.add_pomdp(POMDP(T00, input_names=['a'], state_name='s'))

T0 = np.array([[1, 0],
               [0, 1]]);
T1 = np.array([[0, 1],
               [0, 1]]);
network.add_pomdp(POMDP([T0, T1], input_names=['l'], state_name='q'))
Ms._state_space[-1] = (np.nan,np.nan)
# (Ms._state_space[s][0] > 4)

network.add_connection(['s'], 'l', lambda s: set([1]) if (Ms._state_space[s][0] > 4) else set([0]) )
accept = np.zeros((T00.shape[1],2))
accept[:,1] = 1

val_list, pol_list = solve_reach(network, accept)
Robotmdp = diagonal(get_T_uxXz(network.pomdps['s']), 2, 3)
Robotdfa = diagonal(get_T_uxXz(network.pomdps['q']), 2, 3)
conn = network.connections[0][2].transpose()

t=time.time()
delta = 0.01
# reach_prob, val2 = solve_delta(self.mdp1, self.dfa1, conn, delta, s0=0, q0=0, q_target=1)
strat = np.array([[0] * (Robotmdp.shape[2]), [1] * (Robotmdp.shape[2])]).transpose()
strat = strat[conn]

model, reach_prob = solve_ltl(Robotmdp.todense(), Robotdfa.todense(), strat, delta, s0=0, q0=0, q_target=1)
# np.testing.assert_almost_equal(reach_prob['primal objective'], val_list[0][0, 0], decimal=5)
print(reach_prob)
print([reach_prob['primal objective']])
print([time.time()-t])


In [ ]:
import scipy.sparse
import matplotlib.pyplot as plt

m = scipy.sparse.coo_matrix((reach_prob['x'],(np.array(reach_prob['indices'])[::,1],np.array(reach_prob['indices'])[::,2])))
Pol = m.argmax(1)
Z = m.sum(1)
Z = Z[:-1:].reshape(len(srep[0]),len(srep[1]))
Pol = Pol[:-1:].reshape(len(srep[0]),len(srep[1]))


fig, (ax0, ax1) = plt.subplots(2, 1)

ax0.pcolorfast((-10,10),(-10,10),np.array(Pol))
ax1.pcolorfast((-10,10),(-10,10),np.array(Z))
plt.show()